In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import psycopg2

In [2]:
target_group_id = 1

In [3]:
df = pd.read_excel('Modelos_defunciones_meses.xlsx', 'nov_dic_ene', dtype={'CVEGEO': str})

df

,CVEGEO,nom_ent,nom_mun,pobtot_1,P_60YMAS,def_nov,def_dic,def_ene,score_nov,ventil_nov,...,prob_dic,p_dic+(p_dic-p_nov),pronostico ene,Unnamed: 16,Unnamed: 17,Etiquetas de fila,score_nov.1,prob_nov.1,Score_dic.1,prob_dic.1
0,1001,Aguascalientes,Aguascalientes,948990,102987,206,171,125,17.589041,D20,...,0.001530,0.000839,86.454470,NaN,NaN,D01,-58.929446,0.000055,-64.487428,0.000082
1,1002,Aguascalientes,Asientos,51536,4697,3,4,3,-12.970989,D07,...,0.000573,0.000686,3.223119,NaN,NaN,D02,-41.252814,0.000139,-44.827027,0.000133
2,1003,Aguascalientes,Calvillo,58250,7829,5,2,3,-2.860330,D09,...,0.000825,0.001104,8.646370,NaN,NaN,D03,-33.942486,0.000184,-36.356351,0.000222
3,1004,Aguascalientes,Cosío,17000,1613,5,0,0,-18.461754,D06,...,0.000408,0.000469,0.756107,NaN,NaN,D04,-27.367738,0.000230,-28.769154,0.000279
4,1005,Aguascalientes,Jesús María,129929,9815,7,6,4,5.259145,D12,...,0.001030,0.001472,14.445284,NaN,NaN,D05,-21.745225,0.000309,-21.984820,0.000408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2464,32054,Zacatecas,Villa Hidalgo,19446,2126,4,1,0,-20.198300,D05,...,0.000408,0.000508,1.079783,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2465,32055,Zacatecas,Villanueva,31558,5638,16,7,2,-9.515222,D08,...,0.000573,0.000697,3.928006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2466,32056,Zacatecas,Zacatecas,149607,18847,68,47,39,14.458408,D19,...,0.001456,0.001845,34.771871,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,32057,Zacatecas,Trancoso,20455,1785,6,3,2,-18.784424,D06,...,0.000488,0.000627,1.119629,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df[['CVEGEO', 'P_60YMAS']]
df['CVEGEO'] = df['CVEGEO'].apply(lambda x: x.rjust(5, '0'))
df['gridid_state'] = df['CVEGEO'].str.slice(0, 2)


df

/home/pedro/anaconda3/envs/geo-environment/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pedro/anaconda3/envs/geo-environment/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,CVEGEO,P_60YMAS,gridid_state
0,01001,102987,01
1,01002,4697,01
2,01003,7829,01
3,01004,1613,01
4,01005,9815,01
...,...,...,...
2464,32054,2126,32
2465,32055,5638,32
2466,32056,18847,32
2467,32057,1785,32


In [5]:
df_target_states = df.groupby(['gridid_state']).sum()
df_target_states = df_target_states.reset_index()
df_target_states = df_target_states.rename(columns={'nom_ent': 'name', 'P_60YMAS': 'population'})

df_target_states

,gridid_state,population
0,01,145376
1,02,379302
2,03,75608
3,04,102616
4,05,348537
5,06,91557
6,07,507757
7,08,419821
8,09,1491619
9,10,209478


In [6]:
db_name = os.environ['DBNAME']
db_user = os.environ['DBUSER']
db_port = os.environ['DBPORT']
db_host = os.environ['DBHOST']
db_pass = os.environ['DBPASS']
str_conn = 'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}'.format(db_name=db_name,
                                                                                           db_user=db_user,
                                                                                           db_port=db_port,
                                                                                           db_host=db_host,
                                                                                           db_pass=db_pass)
engine = create_engine(str_conn)

In [7]:
df_states = pd.read_sql('SELECT id, gridid_state FROM mesh_state', con=engine)

df_states

,id,gridid_state
0,1,01
1,2,02
2,3,03
3,4,04
4,5,05
5,6,06
6,7,07
7,8,08
8,9,09
9,10,10


In [8]:
df_target_states['target_group_id'] = pd.Series([1 for i in range(df_states.shape[0])])
df_target_states = df_target_states.merge(df_states, on='gridid_state')
df_target_states = df_target_states.rename(columns={'id': 'state_id'})
df_target_states = df_target_states.drop(columns=['gridid_state'])

df_target_states

,population,target_group_id,state_id
0,145376,1,1
1,379302,1,2
2,75608,1,3
3,102616,1,4
4,348537,1,5
5,91557,1,6
6,507757,1,7
7,419821,1,8
8,1491619,1,9
9,209478,1,10


In [9]:
df_target_states.to_sql('target_group_population_state', con=engine, if_exists='append', index=False)

In [10]:
df_mun = pd.read_sql('SELECT id, gridid_mun FROM mesh_mun', con=engine)

df_mun

,id,gridid_mun
0,1,01001
1,2,01002
2,3,01003
3,4,01004
4,5,01005
...,...,...
2464,2465,32054
2465,2466,32055
2466,2467,32056
2467,2468,32057


In [11]:
df_target_mun = df[['CVEGEO', 'P_60YMAS']]
df_target_mun['CVEGEO'] = df_target_mun['CVEGEO'].apply(lambda x: x.rjust(5, '0'))
df_target_mun['gridid_state'] = df_target_mun['CVEGEO'].str.slice(0, 2)
df_target_mun = df_target_mun.rename(columns={'P_60YMAS': 'population', 'CVEGEO': 'gridid_mun'})

df_target_mun['target_group_id'] = pd.Series([1 for i in range(df_target_mun.shape[0])])
df_target_mun = df_target_mun.merge(df_states, on='gridid_state')
df_target_mun = df_target_mun.rename(columns={'id': 'state_id'})
df_target_mun = df_target_mun.drop(columns=['gridid_state'])

df_target_mun = df_target_mun.merge(df_mun, on='gridid_mun')
df_target_mun = df_target_mun.drop(columns=['gridid_mun'])
df_target_mun = df_target_mun.rename(columns={'id': 'mun_id'})

df_target_mun

/home/pedro/anaconda3/envs/geo-environment/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,population,target_group_id,state_id,mun_id
0,102987,1,1,1
1,4697,1,1,2
2,7829,1,1,3
3,1613,1,1,4
4,9815,1,1,5
...,...,...,...,...
2464,2126,1,32,2465
2465,5638,1,32,2466
2466,18847,1,32,2467
2467,1785,1,32,2468


In [12]:
df_target_mun.to_sql('target_group_population_mun', con=engine, if_exists='append', index=False)